# Results replication: Table 2 and Figures 1/2/3/4/6/9/10
Этот ноутбук строит **SSS (stochastic steady state)** и **эргодические распределения/переходы** строго по логике статьи.

**Важно:**
- *SSS* считаются решением стационарной системы при нулевых инновациях, но с марковскими переходами режимов.
- *Эргодические распределения* считаются длинной симуляцией обученной политики.
- Для некоторых контрфактов (например, Figure 4) требуются **переобученные** веса под альтернативную калибровку; ниже есть ячейки, которые это делают и сохраняют артефакты.

In [ ]:
import os, sys, json
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

sys.path.append('..')

from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer, simulate_paths
from src.steady_states import (
    solve_flexprice_sss,
    solve_commitment_sss_switching,
    solve_discretion_sss_switching,
    solve_taylor_sss,
    export_rbar_tensor,
    commitment_init_from_sss,
)
from src.io_utils import ensure_dir, load_torch, save_csv, save_json

torch.set_num_threads(1)
DEVICE = 'cpu'


In [ ]:
import matplotlib.pyplot as plt

# ---- plotting / saving utils (paper-style) ----
RESULTS_DIR = os.path.join(artifacts_root, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)

def pct(x):
    return 100.0 * np.asarray(x)

def output_gap_pct(y, y_flex):
    y = np.asarray(y); y_flex = np.asarray(y_flex)
    return 100.0 * (y / y_flex - 1.0)

def save_fig(fig, name):
    path = os.path.join(RESULTS_DIR, f'{name}.png')
    fig.savefig(path, dpi=200, bbox_inches='tight')
    print('saved:', path)

plt.rcParams.update({
    'figure.dpi': 120,
    'savefig.dpi': 200,
    'axes.grid': True,
    'grid.alpha': 0.25,
    'axes.spines.top': False,
    'axes.spines.right': False,
})


## Load parameters and trained policies

In [ ]:
from src.io_utils import find_latest_run_dir, load_selected_run, load_torch

params = ModelParams(device=DEVICE, dtype=torch.float32)
artifacts_root = '../artifacts'

POLICIES = {
    'taylor':        {'d_in':5, 'd_out':8},
    'mod_taylor':    {'d_in':5, 'd_out':8},
    'discretion':    {'d_in':5, 'd_out':11},
    'commitment':    {'d_in':7, 'd_out':13},
}

# If you want fully reproducible results, set RUN_DIR explicitly.
# Otherwise, we try (1) selected_runs.json pointer, then (2) most recent run.
RUN_DIR = None  # e.g. '../artifacts/runs/taylor/20260212_101010_cpu16gb_plan_seed123_ab12cd34ef'


def resolve_run_dir(policy: str) -> str:
    if RUN_DIR is not None:
        return RUN_DIR
    sel = load_selected_run(artifacts_root, policy)
    if sel is not None:
        return sel
    latest = find_latest_run_dir(artifacts_root, policy)
    if latest is None:
        raise FileNotFoundError(f'No run directory found for policy={policy}. Train first.')
    return latest


def load_net(policy: str, cfg: TrainConfig) -> PolicyNetwork:
    run_dir = resolve_run_dir(policy)
    # Prefer best checkpoint if available, else final.
    cand = [
        os.path.join(run_dir, getattr(cfg, 'best_weights_name', 'weights_best.pt')),
        os.path.join(run_dir, 'weights_final.pt'),
        os.path.join(run_dir, 'weights.pt'),
    ]
    wpath = None
    for p in cand:
        if os.path.exists(p):
            wpath = p
            break
    if wpath is None:
        raise FileNotFoundError(f'Weights not found in {run_dir}. Looked for: {cand}')

    d_in = POLICIES[policy]['d_in']
    d_out = POLICIES[policy]['d_out']
    net = PolicyNetwork(d_in=d_in, d_out=d_out, hidden=cfg.hidden_layers, activation=cfg.activation)
    net.load_state_dict(load_torch(wpath, map_location='cpu'))
    net.eval()
    return net


## Compute SSS (Table 2 building blocks)
Мы считаем:
- flex-price SSS (natural rates / natural output)
- commitment SSS (switching)
- discretion SSS (switching)
- Taylor SSS по формуле (25) (использует natural rates из flex-price SSS)
- modified Taylor SSS (инфляция ≈ 0 по конструкции, при r̄=r*).

In [ ]:
params64 = ModelParams(device=DEVICE, dtype=torch.float64)

flex = solve_flexprice_sss(params64)
commit_sss = solve_commitment_sss_switching(params64)
disc_sss = solve_discretion_sss_switching(params64)
taylor_sss = solve_taylor_sss(params64, flex)          # dict by regime

# modified Taylor: π target 0, intercept r̄_t = r*_n, so π_sss ~ 0; rates match natural rates.
mod_taylor_sss = {
    0: {'pi': 0.0, 'r': float(flex.by_regime[0]['r_star'])},
    1: {'pi': 0.0, 'r': float(flex.by_regime[1]['r_star'])},
}

print('Flex r*:', flex.by_regime[0]['r_star'], flex.by_regime[1]['r_star'])
print('Taylor π_sss:', taylor_sss[0]['pi'], taylor_sss[1]['pi'])
print('Commitment π_sss:', commit_sss.by_regime[0]['pi'], commit_sss.by_regime[1]['pi'])
print('Discretion π_sss:', disc_sss.by_regime[0]['pi'], disc_sss.by_regime[1]['pi'])


### Table 2
Ниже формируем таблицу SSS по режимам для каждой политики.
Колонки: π (инфляция), r (реальная ставка), i (номинальная ставка), c, y, output gap x.

**Output gap**: \(x = y - y^{flex}\) в том же режиме SSS.

In [ ]:
def fisher_i(r, pi):
    return (1.0 + r) * (1.0 + pi) - 1.0

def table_row(name, by_regime):
    rows = []
    for s in [0,1]:
        d = by_regime[s]
        pi = float(d.get('pi', np.nan))
        r  = float(d.get('r', np.nan))
        i  = float(d.get('i', fisher_i(r, pi)))
        c  = float(d.get('c', np.nan))
        y  = float(d.get('y', np.nan))        chat = float(flex.by_regime[0].get('c', np.nan))
        x  = 100.0*(np.log(c) - np.log(chat))
        rows.append({'policy':name, 'regime':s, 'pi':pi, 'r':r, 'i':i, 'c':c, 'y':y, 'x':x})
    return rows

sss_map = {
    'flex':       {0:flex.by_regime[0], 1:flex.by_regime[1]},
    'taylor':     taylor_sss,
    'mod_taylor': mod_taylor_sss,
    'discretion': {0:disc_sss.by_regime[0], 1:disc_sss.by_regime[1]},
    'commitment': {0:commit_sss.by_regime[0], 1:commit_sss.by_regime[1]},
}

rows=[]
for k,v in sss_map.items():
    rows += table_row(k, v)

table2 = pd.DataFrame(rows)
table2


In [ ]:
ensure_dir(os.path.join(artifacts_root, 'results'))
save_csv(os.path.join(artifacts_root, 'results', 'table2_sss.csv'), table2)
print('Saved:', os.path.join(artifacts_root, 'results', 'table2_sss.csv'))


## Helpers: regime-filtered histograms and transitions

In [ ]:
def hist_by_regime(ax, data, regimes, bins=80, title=''):
    data0 = data[regimes==0]
    data1 = data[regimes==1]
    ax.hist(data0, bins=bins, alpha=0.6, label='normal (s=0)')
    ax.hist(data1, bins=bins, alpha=0.6, label='bad (s=1)')
    ax.set_title(title)
    ax.legend()

def regime_masked_mean(x, s, val):
    out={}
    for r in [0,1]:
        m = (s==r)
        out[r]=float(x[m].mean()) if m.any() else float('nan')
    return out


## Figure 1: Ergodic distribution under commitment
Длинная симуляция обученной commitment-политики, гистограммы по режимам.

In [ ]:
commit_init = commitment_init_from_sss(params64)  # dict {0:{vp,rp},1:{vp,rp}}
cfg_sim = TrainConfig(seed=123, artifacts_root=artifacts_root)
netC = nets['commitment']
trainerC = Trainer(params=params, cfg=cfg_sim, policy='commitment', net=netC, gh_n=7)

x0 = trainerC.simulate_initial_state(B=4096, commitment_sss=commit_init)
paths = simulate_paths(params, 'commitment', netC, T=200_000, burn_in=5_000, x0=x0, compute_implied_i=True)

s = paths['s'].reshape(-1)
pi = pct(paths['pi'].reshape(-1))  # percent
r  = ( (1+paths['i'].reshape(-1))/(1+paths['pi'].reshape(-1)) - 1.0 )
Delta = paths['Delta'].reshape(-1)

fig, axs = plt.subplots(2,2, figsize=(12,8))
hist_by_regime(axs[0,0], pi, s, title='Inflation (%, quarterly)')
hist_by_regime(axs[0,1], r,  s, title='Real rate (%, quarterly)')
hist_by_regime(axs[1,0], Delta, s, title='Price dispersion Δ')

# Output gap proxy at SSS regime: y - y_flex_sss(s)
y = paths['y'].reshape(-1)
yflex0 = float(flex.by_regime[0]['y']); yflex1 = float(flex.by_regime[1]['y'])
yflex = np.where(s==0, yflex0, yflex1)
xgap = output_gap_pct(y, yflex)
hist_by_regime(axs[1,1], xgap, s, title='Output gap (%, vs flex SSS)')
plt.tight_layout(); save_fig(fig, 'Figure1_commitment_ergodic'); plt.show()


## Figure 2: Transition normal → bad (commitment vs discretion)
Стартуем из SSS нормального режима, затем принудительно фиксируем переход в bad и держим режим bad.
Инновации непрерывных шоков = 0, чтобы получить чистую реакцию на смену режима.

In [ ]:

def deterministic_transition(params, policy, net, s_path, x_start, rbar_by_regime=None, T=80):
    """Deterministic transition with eps=0 and exogenous regime path s_path (length T, values 0/1).

    Stores π_t (%), Δ_t (level), output gap x_t (%), defined as 100·(log(c_t) − log(ĉ_t)), implied/ruled nominal i_t (%) and real r_t (%).
    For optimal policies (discretion/commitment), i_t is implied from the (deterministic) Euler equation:
        1+i_t = (λ_t/(β λ_{t+1})) (1+π_{t+1}).
    """
    params0 = ModelParams(**{**params.__dict__})
    params0 = params0.to_torch()
    cfg0 = TrainConfig(seed=0)
    trainer = Trainer(params=params0, cfg=cfg0, policy=policy, net=net, gh_n=7, rbar_by_regime=rbar_by_regime)

    x = x_start.clone()
    out_store = {k: [] for k in ['pi','Delta','xgap','i','r']}

    for t in range(T):
        out = trainer._policy_outputs(x)
        st = __import__('src.model_common', fromlist=['unpack_state']).unpack_state(x, policy)
        ids = __import__('src.model_common', fromlist=['identities']).identities(params0, st, out)

        # forced next regime and zero shocks
        epsA = torch.zeros_like(st.logA)
        epsg = torch.zeros_like(st.loggtilde)
        epst = torch.zeros_like(st.xi)
        s_next = torch.full_like(st.s, int(s_path[min(t, len(s_path)-1)]))
        logA_n, logg_n, xi_n, s_n = __import__('src.model_common', fromlist=['shock_laws_of_motion']).shock_laws_of_motion(
            params0, st, epsA, epsg, epst, s_next
        )

        # build x_{t+1}
        if policy == 'commitment':
            vp_next = out['vartheta'] * out['c'].pow(params0.gamma)
            rp_next = out['varrho'] * out['c'].pow(params0.gamma)
            x_next = torch.stack([out['Delta'], logA_n, logg_n, xi_n, s_n.to(out['Delta'].dtype), vp_next, rp_next], dim=-1)
        else:
            x_next = torch.stack([out['Delta'], logA_n, logg_n, xi_n, s_n.to(out['Delta'].dtype)], dim=-1)

        # next-period objects (for implied i, r)
        out_n = trainer._policy_outputs(x_next)

        # nominal i_t
        if policy == 'taylor':
            i_t = __import__('src.policy_rules', fromlist=['i_taylor']).i_taylor(params0, out['pi'])
        elif policy == 'mod_taylor':
            i_t = __import__('src.policy_rules', fromlist=['i_modified_taylor']).i_modified_taylor(params0, out['pi'], rbar_by_regime, st.s)
        else:
            # deterministic Euler-implied nominal rate
            i_t = (out['lam'] / (params0.beta * out_n['lam'])) * (1.0 + out_n['pi']) - 1.0

        r_t = (1.0 + i_t) / (1.0 + out_n['pi']) - 1.0

        # output gap (paper): x_t = log(c_t) − log(ĉ_t), where ĉ_t is consumption in efficient (flex-price) allocation
        chat = float(flex.by_regime[0]['c'])  # efficient allocation is not affected by cost-push regimes
        chat_t = torch.full_like(out['c'], chat)
        xgap = 100.0 * (torch.log(out['c']) - torch.log(chat_t))

        out_store['pi'].append((100.0*out['pi']).detach().cpu().numpy())
        out_store['Delta'].append(out['Delta'].detach().cpu().numpy())
        out_store['xgap'].append(xgap.detach().cpu().numpy())
        out_store['i'].append((100.0*i_t).detach().cpu().numpy())
        out_store['r'].append((100.0*r_t).detach().cpu().numpy())

        x = x_next

    # average across B
    for k in out_store:
        out_store[k] = np.asarray(out_store[k]).mean(axis=1).squeeze()
    return out_store

# start from SSS by policy
def x0_from_sss_commit(params, s):
    d = commit_sss.by_regime[s]
    return torch.tensor([[d['Delta'], d['logA'], d['loggtilde'], d['xi'], float(s), d['vartheta_prev'], d['varrho_prev']]], dtype=torch.float32)

def x0_from_sss_basic(sss_obj, s):
    d = sss_obj.by_regime[s]
    return torch.tensor([[d['Delta'], d['logA'], d['loggtilde'], d['xi'], float(s)]], dtype=torch.float32)

# Figure 2: normal → bad (commitment vs discretion)
s_path = [0]*1 + [1]*79
trC = deterministic_transition(params, 'commitment', nets['commitment'], s_path, x0_from_sss_commit(params,0), T=80)
trD = deterministic_transition(params, 'discretion', nets['discretion'], s_path, x0_from_sss_basic(disc_sss,0), T=80)

t = np.arange(80)
fig, axs = plt.subplots(2,2, figsize=(12,8))
axs[0,0].plot(t, trC['pi'], label='commitment')
axs[0,0].plot(t, trD['pi'], '--', label='discretion')
axs[0,0].set_title('Inflation π (%, quarterly)'); axs[0,0].legend()

axs[0,1].plot(t, trC['xgap'], label='commitment')
axs[0,1].plot(t, trD['xgap'], '--', label='discretion')
axs[0,1].set_title('Output gap x (%, vs flex SSS)'); axs[0,1].legend()

axs[1,0].plot(t, trC['r'], label='commitment')
axs[1,0].plot(t, trD['r'], '--', label='discretion')
axs[1,0].set_title('Real rate r (%, quarterly)'); axs[1,0].legend()

axs[1,1].plot(t, trC['Delta'], label='commitment')
axs[1,1].plot(t, trD['Delta'], '--', label='discretion')
axs[1,1].set_title('Price dispersion Δ (level)'); axs[1,1].legend()

plt.tight_layout(); save_fig(fig, 'Figure2_transition_normal_to_bad'); plt.show()


## Figure 3: Persistent (regime switch) vs temporary (AR(1) cost-push)
Persistent = переключение режима; temporary = одноразовая инновация ε^τ при сохранении режима normal.
Обе траектории под commitment.

In [ ]:

def temporary_costpush_irf(params, net, eps_tau0=1.0, rho_tau=None, T=80):
    """Temporary AR(1) cost-push innovation in normal regime under commitment.
    eps_tau0 is in *standard-normal* units; xi_{t+1} uses sigma_tau * eps_tau0.
    """
    params_tmp = ModelParams(**{**params.__dict__})
    if rho_tau is not None:
        params_tmp.rho_tau = float(rho_tau)
    params_tmp = params_tmp.to_torch()
    cfg0 = TrainConfig(seed=0)
    tr = Trainer(params=params_tmp, cfg=cfg0, policy='commitment', net=net, gh_n=7)

    x = x0_from_sss_commit(params,0)
    out_store = {'pi':[], 'Delta':[], 'xgap':[], 'r':[]}

    for t in range(T):
        out = tr._policy_outputs(x)
        st = __import__('src.model_common', fromlist=['unpack_state']).unpack_state(x, 'commitment')
        ids = __import__('src.model_common', fromlist=['identities']).identities(params_tmp, st, out)

        # shocks: only epst at t=0
        epsA = torch.zeros(1, dtype=torch.float32)
        epsg = torch.zeros(1, dtype=torch.float32)
        epst = torch.tensor([float(eps_tau0) if t==0 else 0.0], dtype=torch.float32)
        s_next = torch.zeros(1, dtype=torch.long)  # keep normal
        logA_n, logg_n, xi_n, s_n = __import__('src.model_common', fromlist=['shock_laws_of_motion']).shock_laws_of_motion(
            params_tmp, st, epsA, epsg, epst, s_next
        )

        vp = out['vartheta'] * out['c'].pow(params_tmp.gamma)
        rp = out['varrho'] * out['c'].pow(params_tmp.gamma)
        x_next = torch.stack([out['Delta'], logA_n, logg_n, xi_n, s_n.to(out['Delta'].dtype), vp, rp], dim=-1)

        out_n = tr._policy_outputs(x_next)
        i_t = (out['lam'] / (params_tmp.beta * out_n['lam'])) * (1.0 + out_n['pi']) - 1.0
        r_t = (1.0 + i_t) / (1.0 + out_n['pi']) - 1.0

        yflex0 = float(flex.by_regime[0]['y'])
        xgap = 100.0 * (ids['y'] / yflex0 - 1.0)

        out_store['pi'].append(100.0*out['pi'].item())
        out_store['Delta'].append(out['Delta'].item())
        out_store['xgap'].append(float(xgap.item()))
        out_store['r'].append(100.0*float(r_t.item()))

        x = x_next

    return {k: np.array(v) for k,v in out_store.items()}

# Persistent: regime switch to bad and stay
s_path = [0]*1 + [1]*79
pers = deterministic_transition(params, 'commitment', nets['commitment'], s_path, x0_from_sss_commit(params,0), T=80)

# Temporary: one-sigma cost-push innovation (standard-normal units)
temp = temporary_costpush_irf(params, nets['commitment'], eps_tau0=1.0, rho_tau=float(params.rho_tau), T=80)

t = np.arange(80)
fig, axs = plt.subplots(2,2, figsize=(12,8))
axs[0,0].plot(t, pers['pi'], label='persistent (regime)')
axs[0,0].plot(t, temp['pi'], '--', label='temporary (AR1)')
axs[0,0].set_title('Inflation π (%, quarterly)'); axs[0,0].legend()

axs[0,1].plot(t, pers['xgap'], label='persistent')
axs[0,1].plot(t, temp['xgap'], '--', label='temporary')
axs[0,1].set_title('Output gap x (%, vs flex SSS)'); axs[0,1].legend()

axs[1,0].plot(t, pers['r'], label='persistent')
axs[1,0].plot(t, temp['r'], '--', label='temporary')
axs[1,0].set_title('Real rate r (%, quarterly)'); axs[1,0].legend()

axs[1,1].plot(t, pers['Delta'], label='persistent')
axs[1,1].plot(t, temp['Delta'], '--', label='temporary')
axs[1,1].set_title('Dispersion Δ'); axs[1,1].legend()

plt.tight_layout(); save_fig(fig, 'Figure3_persistent_vs_temporary'); plt.show()


## Figure 4: Persistence comparison (ρτ = 0.90 vs 0.99)
Статья сравнивает IRF при разной персистентности AR(1) cost-push.
Чтобы сделать **строго**, нужно **переобучить commitment** под альтернативный `rho_tau=0.99` (и/или `rho_tau=0.90`).

Ниже код делает:
- если веса для `commitment_rhotau099/` существуют — грузит,
- иначе запускает обучение и сохраняет веса.

⚠️ Это может быть долго на CPU; но это единственный строгий путь.

In [ ]:
def ensure_commitment_for_rho_tau(rho_tau: float, tag: str, steps: int = 20000):
    subdir = os.path.join(artifacts_root, tag)
    ensure_dir(subdir)
    wpath = os.path.join(subdir, 'weights.pt')
    if os.path.exists(wpath):
        net = PolicyNetwork(d_in=7, d_out=13, hidden=cfg_default.hidden_layers, activation=cfg_default.activation)
        net.load_state_dict(load_torch(wpath, map_location=DEVICE))
        net.to(DEVICE).eval()
        return net
    # train
    p = ModelParams(device=DEVICE, dtype=torch.float32)
    p.rho_tau = rho_tau
    cfg = TrainConfig(seed=123, steps=steps, batch_size=512, lr=1e-5, grad_clip=1.0, artifacts_root=artifacts_root)
    net = PolicyNetwork(d_in=7, d_out=13, hidden=cfg.hidden_layers, activation=cfg.activation)
    commit_init = commitment_init_from_sss(ModelParams(device=DEVICE, dtype=torch.float64, rho_tau=rho_tau))
    tr = Trainer(params=p, cfg=cfg, policy='commitment', net=net, gh_n=7)
    losses = tr.train(B=2048, commitment_sss=commit_init, fine_tune_float64=True)
    torch.save(net.state_dict(), wpath)
    save_csv(os.path.join(subdir,'train_log.csv'), pd.DataFrame({'loss':losses}))
    return net

# Example usage (uncomment to run strict replication)
# net_090 = ensure_commitment_for_rho_tau(0.90, 'commitment_rhotau090', steps=20000)
# net_099 = ensure_commitment_for_rho_tau(0.99, 'commitment_rhotau099', steps=20000)


In [ ]:
# Figure 4: compare AR(1) cost-push persistence (ρτ=0.90 vs 0.99) under commitment
# Persistent regime switch baseline
pers = deterministic_transition(params, 'commitment', nets['commitment'], [0]*1+[1]*79, x0_from_sss_commit(params,0), T=80)

temp_090 = temporary_costpush_irf(params, nets['commitment'], eps_tau0=1.0, rho_tau=0.90, T=80)
temp_099 = temporary_costpush_irf(params, nets['commitment'], eps_tau0=1.0, rho_tau=0.99, T=80)

t = np.arange(80)
fig, axs = plt.subplots(2,2, figsize=(12,8))

axs[0,0].plot(t, pers['pi'], label='persistent (regime)')
axs[0,0].plot(t, temp_090['pi'], '--', label='temporary ρτ=0.90')
axs[0,0].plot(t, temp_099['pi'], ':', label='temporary ρτ=0.99')
axs[0,0].set_title('Inflation π (%, quarterly)'); axs[0,0].legend()

axs[0,1].plot(t, pers['xgap'], label='persistent')
axs[0,1].plot(t, temp_090['xgap'], '--', label='ρτ=0.90')
axs[0,1].plot(t, temp_099['xgap'], ':', label='ρτ=0.99')
axs[0,1].set_title('Output gap x (%, vs flex SSS)'); axs[0,1].legend()

axs[1,0].plot(t, pers['r'], label='persistent')
axs[1,0].plot(t, temp_090['r'], '--', label='ρτ=0.90')
axs[1,0].plot(t, temp_099['r'], ':', label='ρτ=0.99')
axs[1,0].set_title('Real rate r (%, quarterly)'); axs[1,0].legend()

axs[1,1].plot(t, pers['Delta'], label='persistent')
axs[1,1].plot(t, temp_090['Delta'], '--', label='ρτ=0.90')
axs[1,1].plot(t, temp_099['Delta'], ':', label='ρτ=0.99')
axs[1,1].set_title('Price dispersion Δ (level)'); axs[1,1].legend()

plt.tight_layout(); save_fig(fig, 'Figure4_persistence_comparison'); plt.show()


## Figure 6: Asymmetry normal→bad vs bad→normal
Две детерминированные траектории под commitment:
- normal→bad
- bad→normal
и сравнение величины отклика π.

In [ ]:

tr_nb = deterministic_transition(params, 'commitment', nets['commitment'], [0]*1+[1]*79, x0_from_sss_commit(params,0), T=80)
tr_bn = deterministic_transition(params, 'commitment', nets['commitment'], [1]*1+[0]*79, x0_from_sss_commit(params,1), T=80)
t=np.arange(80)

fig, axs = plt.subplots(2,2, figsize=(12,8))
axs[0,0].plot(t, tr_nb['pi'], label='normal→bad')
axs[0,0].plot(t, tr_bn['pi'], label='bad→normal')
axs[0,0].set_title('Inflation π (%, quarterly)'); axs[0,0].legend()

axs[0,1].plot(t, tr_nb['xgap'], label='normal→bad')
axs[0,1].plot(t, tr_bn['xgap'], label='bad→normal')
axs[0,1].set_title('Output gap x (%, vs flex SSS)'); axs[0,1].legend()

axs[1,0].plot(t, tr_nb['r'], label='normal→bad')
axs[1,0].plot(t, tr_bn['r'], label='bad→normal')
axs[1,0].set_title('Real rate r (%, quarterly)'); axs[1,0].legend()

axs[1,1].plot(t, tr_nb['Delta'], label='normal→bad')
axs[1,1].plot(t, tr_bn['Delta'], label='bad→normal')
axs[1,1].set_title('Price dispersion Δ (level)'); axs[1,1].legend()

plt.tight_layout(); save_fig(fig, 'Figure6_asymmetry_commitment'); plt.show()


## Figure 9: Taylor vs modified Taylor (ergodic distributions)
Длинная симуляция под taylor и mod_taylor, гистограммы по режимам.

In [ ]:
flex32 = solve_flexprice_sss(ModelParams(device=DEVICE, dtype=torch.float64))
rbar = export_rbar_tensor(ModelParams(device=DEVICE, dtype=torch.float64), flex32).to('cpu').float()

cfg_sim = TrainConfig(seed=123, artifacts_root=artifacts_root)

# Taylor
netT = nets['taylor']
trT = Trainer(params=params, cfg=cfg_sim, policy='taylor', net=netT)
x0T = trT.simulate_initial_state(B=4096)
pathsT = simulate_paths(params, 'taylor', netT, T=200_000, burn_in=5_000, x0=x0T)

# Modified Taylor
netM = nets['mod_taylor']
trM = Trainer(params=params, cfg=cfg_sim, policy='mod_taylor', net=netM, rbar_by_regime=rbar)
x0M = trM.simulate_initial_state(B=4096)
pathsM = simulate_paths(params, 'mod_taylor', netM, T=200_000, burn_in=5_000, x0=x0M, rbar_by_regime=rbar)

def fig9_panel(paths, title_prefix):
    s = paths['s'].reshape(-1)
    pi = pct(paths['pi'].reshape(-1))
    r = pct(( (1+paths['i'].reshape(-1))/(1+paths['pi'].reshape(-1)) - 1.0 ))
    fig, axs = plt.subplots(1,2, figsize=(12,4))
    hist_by_regime(axs[0], pi, s, title=f'{title_prefix}: Inflation π (%, quarterly)')
    hist_by_regime(axs[1], r,  s, title=f'{title_prefix}: Real rate r (%, quarterly)')
    plt.tight_layout(); save_fig(fig, f'Figure9_{title_prefix.replace(" ","_")}'); plt.show()

fig9_panel(pathsT, 'Taylor')
fig9_panel(pathsM, 'Modified Taylor')


## Figure 10: Sensitivity to regime length (Taylor rule)
Варьируем `p21` (выход из bad), пересчитываем natural rates из flex-price SSS и затем Taylor SSS по (25).
Также строим контрфакт `p12→1` (normal длится 1 период).

In [ ]:
def fig10_sensitivity(p21_grid, p12_override=None):
    out=[]
    for p21 in p21_grid:
        p = ModelParams(device=DEVICE, dtype=torch.float64)
        p.p21 = float(p21)
        if p12_override is not None:
            p.p12 = float(p12_override)
        flex_p = solve_flexprice_sss(p)
        tay_p = solve_taylor_sss(p, flex_p)
        out.append({'p21':p21, 'pi0':100.0*tay_p[0]['pi'], 'pi1':100.0*tay_p[1]['pi'], 'r0':100.0*tay_p[0]['r'], 'r1':100.0*tay_p[1]['r']})
    return pd.DataFrame(out)

p21_grid = np.linspace(0.05, 0.9, 25)
df_base = fig10_sensitivity(p21_grid)
df_p12_1 = fig10_sensitivity(p21_grid, p12_override=1.0)

fig = fig = plt.figure(figsize=(8,4))
plt.plot(df_base['p21'], df_base['pi0'], label='normal SSS (baseline p12)')
plt.plot(df_base['p21'], df_base['pi1'], '--', label='bad SSS (baseline p12)')
plt.plot(df_p12_1['p21'], df_p12_1['pi1'], ':', label='bad SSS (p12→1)')
plt.axhline(0.0, linewidth=1)
plt.title('Figure 10: Taylor SSS inflation vs regime length (p21)')
plt.xlabel('p21 (prob. exit bad)'); plt.ylabel('π_sss (%, quarterly)')
plt.legend(); plt.tight_layout(); save_fig(fig, 'Figure10_taylor_sss_inflation'); plt.show()

plt.figure(figsize=(8,4))
plt.plot(df_base['p21'], df_base['r0'], label='normal SSS (baseline)')
plt.plot(df_base['p21'], df_base['r1'], '--', label='bad SSS (baseline)')
plt.plot(df_p12_1['p21'], df_p12_1['r1'], ':', label='bad SSS (p12→1)')
plt.title('Figure 10: Taylor SSS real rate vs regime length (p21)')
plt.xlabel('p21'); plt.ylabel('r_sss (%, quarterly)')
plt.legend(); plt.tight_layout(); save_fig(fig, 'Figure10_taylor_sss_realrate'); plt.show()
